In [1]:
import os
os.chdir('../..')

In [2]:
import logging
import sys

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Avoid duplicate handlers
if not logger.handlers:
    handler = logging.StreamHandler(sys.stdout)  # stdout works better than stderr in Jupyter
    handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
    logger.addHandler(handler)

In [5]:
import numpy as np
import time
import logging
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Enable GPU logging (set True to see device logs)
tf.debugging.set_log_device_placement(False)

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def keras_rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))

def keras_rmsle(y_true, y_pred):
    # Clip predictions to avoid log(0)
    y_true = tf.clip_by_value(y_true, 0.0, np.inf)
    y_pred = tf.clip_by_value(y_pred, 0.0, np.inf)
    
    first_log = tf.math.log1p(y_pred)
    second_log = tf.math.log1p(y_true)
    return tf.sqrt(tf.reduce_mean(tf.square(first_log - second_log)))

keras_rmsle.__name__ = 'rmsle'  # so Keras logs it nicely

keras_rmse.__name__ = 'rmse'  # Required for proper logging in Keras

# Wrapper Class
class KerasNNWrapper:
    def __init__(self, build_fn, metric_fn, name='keras_nn'):
        self.build_fn = build_fn
        self.name = name
        self.metric_fn = metric_fn
        self.models = []
        self.oof_preds = None

    def fit(self, X, y, folds=5, epochs=100, batch_size=1024):
        X = X.astype(np.float32)
        y = y.values.astype(np.float32) if hasattr(y, 'values') else y.astype(np.float32)

        start_time = time.time()
        logger.info(f"Starting training of {self.name} model with {folds} folds")

        self.oof_preds = np.zeros(len(X), dtype=np.float32)
        self.models = []
        kf = KFold(n_splits=folds, shuffle=True, random_state=42)

        # Callbacks
        early_stopping = keras.callbacks.EarlyStopping(
            monitor='val_loss', patience=5, restore_best_weights=True
        )
        lr_schedule = keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr=1e-6
        )
    
        for fold, (train_idx, val_idx) in enumerate(kf.split(X), 1):
            fold_start = time.time()
            logger.info(f"Training {self.name} - Fold {fold}/{folds}")

            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]

            model = self.build_fn()
            model.fit(X_train, y_train,
                      validation_data=(X_val, y_val),
                      epochs=epochs,
                      batch_size=batch_size,
                      verbose=2,
                      callbacks=[lr_schedule])

            preds = model.predict(X_val).flatten()
            self.oof_preds[val_idx] = preds
            fold_score = self.metric_fn(y_val, preds)
            logger.info(f"Fold {fold} score: {fold_score:.4f}")

            self.models.append(model)
            logger.info(f"Completed {self.name} - Fold {fold} in {time.time() - fold_start:.2f} seconds")

        total_score = self.metric_fn(y, self.oof_preds)
        logger.info(f"Out-of-fold score: {total_score:.4f}")
        logger.info(f"Completed training of {self.name} in {time.time() - start_time:.2f} seconds")

    def predict(self, X):
        X = X.astype(np.float32)
        preds = [model.predict(X).flatten() for model in self.models]
        return np.mean(np.column_stack(preds), axis=1)

    def retrain_full(self, X, y, epochs=100, batch_size=1024):
        X = X.astype(np.float32)
        y = y.values.astype(np.float32) if hasattr(y, 'values') else y.astype(np.float32)

        start_time = time.time()
        logger.info(f"Starting full retraining of {self.name} model")

        model = self.build_fn()
        model.fit(X, y, epochs=epochs, batch_size=batch_size, verbose=2)
        self.models = [model]

        logger.info(f"Completed full retraining of {self.name} in {time.time() - start_time:.2f} seconds")


In [6]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

# Transform Sex to 0 and 1
df_train['Sex'] = df_train['Sex'].map({'male': 0, 'female': 1})
df_test['Sex'] = df_test['Sex'].map({'male': 0, 'female': 1})

exclude_cols = ['id', 'Sex', 'Calories']
numeric_cols = [col for col in df_train.columns if col not in exclude_cols]
# Initialize scaler
scaler = StandardScaler()

# Fit and transform training data (excluding categorical columns)
df_train[numeric_cols] = scaler.fit_transform(df_train[numeric_cols])
df_test[numeric_cols] = scaler.transform(df_test[numeric_cols])


# Prepare features and target
X_train = df_train.drop(['Calories', 'id'], axis=1)
y_train = df_train['Calories']

# Convert X_train and y_train to float32
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)
df_test = df_test.astype(np.float32)

In [ ]:
input_dim = X_train.shape[1]

# Keras Model Builder
def build_model(input_dim):
    model = keras.Sequential([
        layers.Input(shape=(input_dim,)),

        layers.Dense(256),
        layers.BatchNormalization(),
        layers.Activation('swish'),
        # layers.Dropout(0.2),

        layers.Dense(128),
        layers.BatchNormalization(),
        layers.Activation('swish'),
        # layers.Dropout(0.2),

        layers.Dense(64),
        layers.BatchNormalization(),
        layers.Activation('swish'),
        # layers.Dropout(0.2),

        layers.Dense(32),
        layers.BatchNormalization(),
        layers.Activation('swish'),

        layers.Dense(1)
    ])

    optimizer = keras.optimizers.Adam()
    model.compile(
        optimizer=optimizer,
        loss=keras_rmsle
    )
    return model

model = KerasNNWrapper(
    build_fn=lambda: build_model(input_dim),
    metric_fn=keras_rmsle,
    name='keras_nn',
    )
model.fit(X_train, y_train, folds=3, epochs=100, batch_size=3_000)


In [ ]:
np.save('nn1_oof_preds.npy', model.oof_preds)

In [ ]:
# Save the 'id' column before dropping it
test_ids = df_test['id']

# Drop 'id' column before prediction
df_test = df_test.drop('id', axis=1)

# Make predictions on test data
test_preds = model.predict(df_test)

# Create submission file
submission = pd.DataFrame({
    'id': test_ids,
    'Calories': test_preds
})

# Save submission file
submission.to_csv('submission_nn1.csv', index=False)